In [1]:
print('hola')

hola


In [2]:
import pandas as pd

# Load Excel file into a DataFrame
sheet_name = 'Sheet1'  # Replace with the actual sheet name if needed
CHS = pd.read_excel('CHS_files.xlsx', sheet_name=sheet_name)
CHE = pd.read_excel('CHE_files.xlsx', sheet_name=sheet_name)
# Specify the column to check for empty records
column_to_check = 'abstract'  # Replace with the actual column name

# Check how many records are empty in the specified column
empty_records_count_che = CHE[column_to_check].isna().sum()
empty_records_count_chs = CHS[column_to_check].isna().sum()

print(f'The number of empty records in the column "{column_to_check}" for CHE file is: {empty_records_count_che} out of {CHE.Label.count()}')
print(f'The number of empty records in the column "{column_to_check}" for CHS file is: {empty_records_count_chs} out of {CHS.Label.count()}')


The number of empty records in the column "abstract" for CHE file is: 376 out of 9788
The number of empty records in the column "abstract" for CHS file is: 1036 out of 10153


In [3]:
non_abstract_CHS = CHS[CHS.abstract.isna()].reset_index()

In [5]:
non_abstract_CHS[non_abstract_CHS.url.isna()]

,index,short_title,abstract,Label,Subgroup,url,PMID,PMCID,Match
0,7,OCCUPATIONAL RESPIRATORY DISEASES,NaN,Relevant,Clinical,NaN,NaN,NaN,title - 0.65
3,17,Studies in sensitization: Influence of occupat...,NaN,Relevant,Clinical,NaN,NaN,NaN,NaN
4,34,Guidelines for Occupational Asthma,NaN,Relevant,Clinical,NaN,NaN,NaN,title - 0.683
6,38,Ulcerative stomatitis in a neonate due to a ch...,NaN,Relevant,Clinical,NaN,NaN,NaN,NaN
7,55,DOMESTIC CHLORINE POISONING,NaN,Relevant,Clinical,NaN,NaN,NaN,title - 0.592
...,...,...,...,...,...,...,...,...,...
1020,9739,THE HOLY PLACES OF ARABIA: THEIR WATER-SUPPLY ...,NaN,Irrelevant,NaN,NaN,NaN,NaN,NaN
1026,9889,"The ear, nose, and throat in school medical in...",NaN,Irrelevant,NaN,NaN,NaN,NaN,title - 0.732
1027,9931,A consideration of the possible effect of cert...,NaN,Irrelevant,NaN,NaN,NaN,NaN,NaN
1031,10054,Hypercalcaemic crisis in a young man with exte...,NaN,Irrelevant,NaN,NaN,NaN,NaN,NaN
